In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

이 튜토리얼은 PyTorch를 사용하여 [Gymnasium](https://gymnasium.farama.org)의 CartPole-v1 태스크에서 Deep Q Learning (DQN) 에이전트를 학습시키는 방법을 보여줍니다.

태스크

에이전트는 카트를 왼쪽 또는 오른쪽으로 움직이는 두 가지 액션 중 하나를 결정해야 합니다. 이를 통해 카트에 부착된 폴이 똑바로 서 있도록 유지해야 합니다. 환경에 대한 더 많은 정보와 다른 도전적인 환경은 [Gymnasium 웹사이트](https://gymnasium.farama.org/environments/classic\_control/cart\_pole/)에서 찾을 수 있습니다.

![CartPole](https://pytorch.org/tutorials/\_static/img/cartpole.gif)

에이전트가 환경의 현재 상태를 관찰하고 행동을 선택하면, 환경은 새로운 상태로 전이되고 행동의 결과를 나타내는 보상을 반환합니다. 이 태스크에서 보상은 매 타임스텝마다 +1이며, 폴이 너무 많이 기울거나 카트가 중심에서 2.4 유닛 이상 멀어지면 환경이 종료됩니다. 이는 더 잘 수행하는 시나리오가 더 오랜 기간 동안 실행되어 더 큰 반환 값을 누적함을 의미합니다.

CartPole 태스크는 에이전트에 대한 입력이 환경 상태(위치, 속도 등)를 나타내는 4개의 실제 값이 되도록 설계되었습니다. 우리는 이러한 4개의 입력을 스케일링 없이 받아들이고, 2개의 출력(각 행동에 대해 하나씩)을 가진 작은 완전 연결 네트워크를 통과시킵니다. 네트워크는 입력 상태가 주어졌을 때 각 행동에 대한 기대 값을 예측하도록 학습됩니다. 그런 다음 기대 값이 가장 높은 행동이 선택됩니다.

패키지

먼저, 필요한 패키지를 가져옵니다. 우선, [pip]{.title-ref}를 사용하여 설치되는 환경을 위해 [gymnasium](https://gymnasium.farama.org/)이 필요합니다. 이는 원래의 OpenAI Gym 프로젝트를 포크한 것이며 Gym v0.19 이후로 동일한 팀에 의해 유지 관리되고 있습니다. Google Colab에서 실행 중인 경우 다음을 실행하세요:

``` {.sourceCode .bash}

%%bash

pip3 install gymnasium[classic_control]

```

또한 PyTorch에서 다음을 사용합니다:

- 신경망 (`torch.nn`)

- 최적화 (`torch.optim`)

- 자동 미분 (`torch.autograd`)

이 튜토리얼에서는 PyTorch를 사용하여 Gymnasium 환경에서 DQN 에이전트를 학습시키는 과정을 설명합니다. 에이전트는 카트의 좌우 이동을 결정하여 폴을 똑바로 세우는 것이 목표입니다. 에이전트는 환경의 상태를 관찰하고 행동을 선택하면, 환경은 새로운 상태로 전이되고 행동의 결과에 대한 보상을 반환합니다.

CartPole 태스크에서는 환경 상태를 나타내는 4개의 실수 값이 에이전트의 입력으로 사용됩니다. 이 입력 값들을 스케일링 없이 작은 완전 연결 신경망에 통과시켜 각 행동에 대한 출력 값을 얻습니다. 신경망은 주어진 상태에서 각 행동의 기대 값을 예측하도록 학습되며, 기대 값이 가장 높은 행동을 선택하게 됩니다.

튜토리얼에 필요한 패키지로는 환경을 위한 Gymnasium과 PyTorch의 신경망, 최적화, 자동 미분 기능 등이 사용됩니다. 이를 통해 DQN 에이전트를 구현하고 학습시킬 수 있습니다.

In [20]:
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

env = gym.make("CartPole-v1",render_mode="human")

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Replay Memory

\=============

우리는 DQN 학습을 위해 경험 리플레이 메모리(experience replay memory)를 사용할 것입니다. 이것은 에이전트가 관찰한 전이(transition)를 저장하여 나중에 이 데이터를 재사용할 수 있도록 합니다. 리플레이 메모리에서 무작위로 샘플링함으로써 배치를 구성하는 전이들은 서로 상관관계가 없어집니다. 이렇게 하면 DQN 학습 과정이 크게 안정화되고 개선된다는 것이 입증되었습니다.

이를 위해 우리는 두 개의 클래스가 필요합니다:

\- `Transition` - 환경에서 단일 전이를 나타내는 명명된 튜플(named tuple)입니다. 본질적으로 (state, action) 쌍을 (next\_state, reward) 결과에 매핑합니다. 여기서 state는 나중에 설명할 화면 차이 이미지입니다.

\- `ReplayMemory` - 최근에 관찰된 전이를 보유하는 제한된 크기의 순환 버퍼입니다. 또한 학습을 위해 무작위로 전이 배치를 선택하는 `.sample()` 메서드를 구현합니다.

경험 리플레이 메모리는 DQN 에이전트의 학습 과정에서 중요한 역할을 합니다. 에이전트가 환경과 상호 작용하면서 관찰한 전이(state, action, next\_state, reward)를 저장하고, 이후 학습 시에 이 데이터를 재사용할 수 있도록 합니다.

리플레이 메모리에서 무작위로 샘플링하여 전이 배치를 구성하면, 배치 내의 전이들 간의 상관관계를 줄일 수 있습니다. 이는 DQN 학습을 안정화하고 개선하는 데 큰 도움이 됩니다.

`Transition` 클래스는 환경에서의 단일 전이를 나타내는 명명된 튜플입니다. (state, action) 쌍을 (next\_state, reward) 결과에 매핑합니다. 여기서 state는 화면 차이 이미지와 같은 형태로 표현될 수 있습니다.

`ReplayMemory` 클래스는 최근에 관찰된 전이를 저장하는 제한된 크기의 순환 버퍼입니다. 새로운 전이가 추가될 때 오래된 전이는 버퍼에서 제거됩니다. 또한, `.sample()` 메서드를 통해 무작위로 전이 배치를 선택하여 학습에 사용할 수 있습니다.

이러한 리플레이 메모리를 활용하여 DQN 에이전트를 학습시키면 경험의 효율적인 재사용과 학습의 안정화를 기대할 수 있습니다.

In [3]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

### DQN 알고리즘

\=============

우리가 다루는 환경은 결정론적이므로, 여기서 제시된 모든 방정식은 단순성을 위해 결정론적으로 공식화되었습니다. 강화 학습 문헌에서는 환경에서의 확률적 전이에 대한 기대값도 포함될 것입니다.

우리의 목표는 할인된 누적 보상 $R\_{t\_0} = \\sum\_{t=t\_0}^{\\infty} \\gamma^{t - t\_0} r\_t$를 최대화하려는 정책을 학습시키는 것입니다. 여기서 $R\_{t\_0}$는 _반환(return)_으로도 알려져 있습니다. 할인율 $\\gamma$는 $0$과 $1$ 사이의 상수로, 합이 수렴되도록 보장합니다. 낮은 $\\gamma$ 값은 먼 미래의 불확실한 보상을 에이전트가 확신할 수 있는 가까운 미래의 보상보다 덜 중요하게 만듭니다. 또한 에이전트가 먼 미래의 동등한 보상보다 시간적으로 더 가까운 보상을 모으도록 장려합니다.

Q-learning의 주요 아이디어는 $Q^\*: State \\times Action \\rightarrow \\mathbb{R}$라는 함수가 있다면, 즉 주어진 상태에서 행동을 취했을 때 얻게 될 반환값을 알려줄 수 있다면, 보상을 최대화하는 정책을 쉽게 구성할 수 있다는 것입니다:

$$\\pi^\*(s) = \\arg\\!\\max\_a \\ Q^\*(s, a)$$

그러나 우리는 세상에 대해 모든 것을 알고 있지 않기 때문에 $Q^\*$에 접근할 수 없습니다. 하지만 신경망은 범용 함수 근사기이므로, 간단히 하나를 만들어 $Q^\*$와 유사하도록 학습시킬 수 있습니다.

학습 업데이트 규칙으로는 어떤 정책에 대한 모든 $Q$ 함수가 벨만 방정식을 따른다는 사실을 이용할 것입니다:

$$Q^{\\pi}(s, a) = r + \\gamma Q^{\\pi}(s', \\pi(s'))$$

등식의 양쪽 간의 차이를 시간차 오류(temporal difference error) $\\delta$라고 합니다:

$$\\delta = Q(s, a) - (r + \\gamma \\max\_a' Q(s', a))$$

이 오류를 최소화하기 위해 \[Huber 손실\](https://en.wikipedia.org/wiki/Huber\_loss)을 사용할 것입니다. Huber 손실은 오류가 작을 때는 평균 제곱 오류(MSE)처럼 작동하고, 오류가 클 때는 평균 절대 오류(MAE)처럼 작동합니다. 이렇게 하면 $Q$ 추정값이 매우 노이즈가 있을 때 이상치에 더 강건해집니다. 우리는 리플레이 메모리에서 샘플링된 전이 배치 $B$에 대해 이를 계산합니다:

$$\\mathcal{L} = \\frac{1}{|B|}\\sum\_{(s, a, s', r) \\ \\in \\ B} \\mathcal{L}(\\delta)$$

$$\\begin{aligned}

\\text{where} \\quad \\mathcal{L}(\\delta) = \\begin{cases}

\\frac{1}{2}{\\delta^2} & \\text{for } |\\delta| \\le 1, \\\\

|\\delta| - \\frac{1}{2} & \\text{otherwise.}

\\end{cases}

\\end{aligned}$$

Q-네트워크

\---------

우리의 모델은 현재와 이전 화면 패치의 차이를 입력으로 받는 피드 포워드 신경망이 될 것입니다. 출력은 $Q(s, \\mathrm{left})$와 $Q(s, \\mathrm{right})$의 두 가지이며 ($s$는 네트워크에 대한 입력), 이는 각각 왼쪽과 오른쪽 행동을 취할 때의 _기대 반환값_을 나타냅니다. 결과적으로 신경망은 현재 입력이 주어졌을 때 각 행동을 취함으로써 얻을 수 있는 기대 반환값을 예측하려고 시도합니다.

DQN 알고리즘은 Q-learning을 기반으로 하며, 신경망을 사용하여 최적의 Q 함수를 근사합니다. 에이전트는 현재 상태에서 가능한 행동들의 Q 값을 예측하는 신경망을 학습시킵니다. 학습 과정에서는 벨만 방정식을 이용하여 시간차 오류를 계산하고, 이를 최소화하는 방향으로 신경망을 업데이트합니다. 손실 함수로는 Huber 손실을 사용하여 이상치에 강건한 학습을 진행합니다.

리플레이 메모리에서 샘플링한 전이 배치를 사용하여 Q 함수를 업데이트하며, 이를 통해 경험을 효율적으로 재사용할 수 있습니다. 최종적으로는 학습된 Q 함수를 이용하여 각 상태에서 Q 값이 최대가 되는 행동을 선택함으로써 최적의 정책을 구할 수 있습니다.

이러한 DQN 알고리즘을 통해 에이전트는 주어진 환경에서 최적의 행동 정책을 학습할 수 있게 됩니다.

In [4]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

### 학습

========

하이퍼파라미터와 유틸리티

-----------------------------

이 부분에서는 모델과 옵티마이저를 인스턴스화하고, 몇 가지 유틸리티 함수를 정의합니다:

- select_action - 엡실론 탐욕 정책에 따라 행동을 선택합니다. 간단히 말해, 때로는 모델을 사용하여 행동을 선택하고, 때로는 균일하게 샘플링할 것입니다. 무작위 행동을 선택할 확률은 EPS_START에서 시작하여 EPS_END를 향해 지수적으로 감소할 것입니다. EPS_DECAY는 감소 속도를 제어합니다.

- plot_durations - 에피소드의 지속 시간을 플롯팅하는 도우미 함수이며, 최근 100개 에피소드의 평균값(공식 평가에서 사용되는 척도)과 함께 표시합니다. 이 플롯은 메인 학습 루프를 포함하는 셀 아래에 위치할 것이며, 매 에피소드 이후 업데이트됩니다.

하이퍼파라미터는 모델의 학습을 제어하는 데 사용되는 사용자 정의 매개변수입니다. 여기서는 엡실론 탐욕 정책의 엡실론 값을 조절하는 하이퍼파라미터들이 정의됩니다. 엡실론 값은 초기에는 높은 값(EPS_START)에서 시작하여 점진적으로 낮은 값(EPS_END)으로 감소하며, 감소 속도는 EPS_DECAY에 의해 제어됩니다. 이를 통해 초기에는 탐험을 많이 하다가 점차 학습이 진행됨에 따라 탐욕적인 행동 선택으로 전환하게 됩니다.

select_action 함수는 현재 상태에서 엡실론 탐욕 정책에 따라 행동을 선택합니다. 엡실론 확률로 무작위 행동을 선택하고, 그렇지 않은 경우에는 모델을 사용하여 Q 값이 가장 높은 행동을 선택합니다.

plot_durations 함수는 에피소드의 지속 시간을 플롯팅하는 도우미 함수입니다. 각 에피소드의 지속 시간과 최근 100개 에피소드의 평균 지속 시간을 함께 표시합니다. 이 플롯을 통해 학습 과정에서 에이전트의 성능 향상을 시각적으로 확인할 수 있습니다.

이러한 하이퍼파라미터와 유틸리티 함수들은 DQN 에이전트의 학습 과정을 제어하고 모니터링하는 데 사용됩니다. 이를 통해 학습의 진행 상황을 파악하고, 하이퍼파라미터를 조정하여 최적의 성능을 달성할 수 있습니다.

In [5]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

# Get number of actions from gym action space
n_actions = env.action_space.n
# Get the number of state observations
state, info = env.reset()
n_observations = len(state)

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1).indices.view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)


episode_durations = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

2024-04-05 02:36:29.961 Python[43671:2369125] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/7b/kknnc53s1w531k3vq6_3br1h0000gn/T/com.apple.python3.savedState


### 학습 루프

\=============

마지막으로, 모델을 학습시키는 코드입니다.

여기서는 최적화의 단일 단계를 수행하는 `optimize_model` 함수를 찾을 수 있습니다. 먼저 배치를 샘플링하고, 모든 텐서를 하나의 텐서로 연결한 다음, $Q(s\_t, a\_t)$와 $V(s\_{t+1}) = \\max\_a Q(s\_{t+1}, a)$를 계산하여 손실 함수에 포함시킵니다. 정의에 따라 $s$가 터미널 상태인 경우 $V(s) = 0$으로 설정합니다. 또한 안정성을 높이기 위해 타겟 네트워크를 사용하여 $V(s\_{t+1})$를 계산합니다. 타겟 네트워크는 이전에 정의된 하이퍼파라미터 `TAU`에 의해 제어되는 \[소프트 업데이트\](https://arxiv.org/pdf/1509.02971.pdf)를 사용하여 매 단계마다 업데이트됩니다.

학습 루프에서는 `optimize_model` 함수를 사용하여 모델을 최적화합니다. 이 함수는 리플레이 메모리에서 배치를 샘플링하고, 현재 상태에서의 Q 값과 다음 상태에서의 최대 Q 값을 계산하여 손실 함수를 구성합니다. 이때 다음 상태가 터미널 상태인 경우에는 해당 상태의 가치를 0으로 설정합니다.

안정적인 학습을 위해 타겟 네트워크를 사용하여 다음 상태의 Q 값을 계산합니다. 타겟 네트워크는 메인 네트워크와 동일한 구조를 가지지만, 가중치 업데이트 속도가 느립니다. 이를 통해 학습 과정에서의 진동을 줄일 수 있습니다. 타겟 네트워크의 가중치는 `TAU` 하이퍼파라미터에 의해 제어되는 소프트 업데이트 방식으로 업데이트됩니다.

학습 루프에서는 각 단계마다 `optimize_model` 함수를 호출하여 모델을 업데이트하고, 에피소드가 종료될 때마다 타겟 네트워크를 업데이트합니다. 또한 일정 간격마다 모델의 성능을 평가하고 결과를 출력합니다.

이러한 학습 루프를 통해 DQN 에이전트는 환경과의 상호작용을 통해 경험을 쌓고, 리플레이 메모리에 저장된 경험을 활용하여 Q 함수를 학습합니다. 최적화 과정을 반복하면서 에이전트는 점진적으로 성능을 향상시키게 됩니다.

In [6]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

학습 루프의 시작에서는 환경을 초기화하고 초기 state 텐서를 얻습니다. 그 다음 행동을 샘플링하고 실행한 후, 다음 상태와 보상(항상 1)을 관찰하고 모델을 한 번 최적화합니다. 에피소드가 끝나면(모델이 실패하면) 루프를 다시 시작합니다.

num_episodes는 GPU를 사용할 수 있는 경우 600으로 설정되고, 그렇지 않은 경우 학습 시간이 너무 오래 걸리지 않도록 50 에피소드로 설정됩니다. 그러나 CartPole 환경에서 좋은 성능을 관찰하기에는 50 에피소드로는 충분하지 않을 수 있습니다. 600 에피소드의 학습 과정에서 모델이 지속적으로 500 단계를 달성하는 것을 확인할 수 있어야 합니다.

강화 학습 에이전트를 학습시키는 과정은 노이즈가 있을 수 있으므로, 수렴이 관찰되지 않는 경우 학습을 다시 시작하면 더 나은 결과를 얻을 수 있습니다.

이러한 학습 루프를 통해 DQN 에이전트는 CartPole 환경에서 최적의 정책을 학습하게 됩니다. 에피소드를 반복하면서 에이전트는 경험을 쌓고, 리플레이 메모리를 활용하여 Q 함수를 업데이트합니다. 학습이 진행됨에 따라 에이전트의 성능은 점진적으로 향상되어, 최종적으로는 CartPole을 오랜 시간 동안 균형 잡힌 상태로 유지할 수 있게 됩니다.

충분한 에피소드로 학습을 진행하고, 필요에 따라 학습을 다시 시작하면서 최적의 성능을 달성할 수 있도록 실험해 보는 것이 중요합니다.

In [8]:
if torch.cuda.is_available():
    num_episodes = 10000
else:
    num_episodes = 5000

for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

KeyboardInterrupt: 


![](https://pytorch.org/tutorials/_static/img/reinforcement_learning_diagram.jpg)


먼저 에이전트는 현재 상태에 대해 정책(Policy)에 따라 행동을 선택하거나 무작위로 행동을 선택합니다. 선택된 행동은 gym 환경에 전달되어 다음 상태와 보상을 받아옵니다.

이 결과는 리플레이 메모리(Replay Memory)에 저장되며, 매 반복마다 최적화 단계(Optimization Step)가 실행됩니다. 최적화 단계에서는 리플레이 메모리에서 무작위로 배치(Batch)를 선택하여 새로운 정책을 학습합니다. 이때 "이전" 타겟 네트워크(Target Network)가 사용되어 기대 Q 값을 계산합니다.

학습 과정에서 현재 정책 네트워크(Policy Network)와 타겟 네트워크는 서로 다른 역할을 합니다. 정책 네트워크는 현재 상태에서의 행동을 선택하고, 타겟 네트워크는 다음 상태에서의 Q 값을 예측하는 데 사용됩니다. 이는 학습의 안정성을 높이기 위한 방법으로, 타겟 네트워크의 가중치는 일정 간격마다 정책 네트워크의 가중치로부터 소프트 업데이트됩니다.

매 단계마다 정책 네트워크는 리플레이 메모리에서 샘플링된 경험을 기반으로 학습되며, 손실 함수를 최소화하는 방향으로 가중치가 업데이트됩니다. 이를 통해 에이전트는 점진적으로 최적의 정책을 학습하게 됩니다.

다이어그램에서는 이러한 과정이 반복되는 것을 볼 수 있습니다. 에이전트는 환경과 상호작용하며 경험을 쌓고, 리플레이 메모리에 저장된 경험을 활용하여 학습을 수행합니다. 학습이 진행됨에 따라 에이전트의 성능은 향상되고, 최적의 정책에 수렴하게 됩니다.

전체적인 데이터 흐름은 다음과 같이 요약할 수 있습니다:
1. 에이전트가 현재 상태에서 행동을 선택합니다.
2. 선택된 행동을 환경에 전달하여 다음 상태와 보상을 받아옵니다.
3. 현재 상태, 행동, 보상, 다음 상태를 리플레이 메모리에 저장합니다.
4. 리플레이 메모리에서 무작위로 배치를 선택하여 정책 네트워크를 학습합니다.
5. 타겟 네트워크를 사용하여 기대 Q 값을 계산하고, 정책 네트워크의 손실 함수를 최소화하는 방향으로 가중치를 업데이트합니다.
6. 일정 간격마다 타겟 네트워크의 가중치를 정책 네트워크의 가중치로부터 소프트 업데이트합니다.

이러한 과정을 반복하면서 에이전트는 최적의 정책을 학습하게 됩니다.


In [17]:
!pip3 install gym_super_mario_bros nes_py
!pip3 install torchvision
!pip3 install --upgrade Pillow


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


### 실습 훈련 : DQN으로 슈퍼마리오 구현하기